In [ ]:
import os
import time
import datetime

import numpy as np
import pystac_client
import dask_image
import planetary_computer as pc

import rioxarray
import mercantile
import xarray as xr
import hvplot.xarray
import geopandas as gpd

from pathlib import Path
from mercantile import Tile
from rasterio.crs import CRS
from functools import partial
from odc.stac import stac_load
from shapely.geometry import box
from geocube.api.core import make_geocube
from typing import Optional, Sequence, List, Tuple, Union, Any
from dask_image.ndfilters import maximum_filter, median_filter


In [ ]:
# PC Configurations

PC_S2COG = {
    "url": "https://planetarycomputer.microsoft.com/api/stac/v1",
    "collections": ["sentinel-2-l2a"],
}

PC_S1RTC = {
    "url": "https://planetarycomputer.microsoft.com/api/stac/v1",
    "collections": ["sentinel-1-rtc"],
}

odc_cfg = {
    "sentinel-2-l2a": {
        "assets": {
            "*": {"data_type": "float64", "nodata": np.nan},
            "SCL": {"data_type": "uint8", "nodata": 0},
            "visual": {"data_type": "uint8", "nodata": 0},
        },
    },
    "sentinel-1-rtc": {
        "assets": {
            "*": {"data_type": "float64", "nodata": np.nan},
            # "SCL": {"data_type": "uint8", "nodata": 0},
            # "visual": {"data_type": "uint8", "nodata": 0},
        },
    },
    "*": {"warnings": "ignore"},
}

pc_client = pystac_client.Client.open(PC_S1RTC["url"])


In [ ]:
# PC Functions

def tile_by_point(coords: Union[List, Tuple], zoom=12):
    tile = mercantile.tile(lng=coords[0], lat=coords[1], zoom=zoom)
    return tile


def items_from_pc(
    bbox: list,
    start_date: str,
    query: Optional[dict] = None,
    end_date: Optional[str] = None,
    collections: Sequence[str] = PC_S2COG["collections"],
):

    end_date = end_date or start_date
    search = pc_client.search(
        bbox=bbox,
        datetime=f"{start_date}/{end_date}",
        collections=collections,
        query=query or {},
    )

    return search.items()


def dataset_from_items(items: List, bands: List[str] = ["vv"], cfg: dict = odc_cfg):

    ds = stac_load(
        items,
        bands=bands,
        # resolution=10,
        chunks={"x": 256, "y": 256, "time": -1},  # <-- use Dask
        patch_url=pc.sign,
        groupby="solar_day",
        # crs="epsg:4326",
        stac_cfg=cfg,
    )

    return ds


def process_tile(tile: Tile):

    bbox = mercantile.bounds(tile)

    end_date = datetime.datetime.today().strftime("%Y-%m-%d")
    start_date = (
        datetime.datetime.today() - datetime.timedelta(days=30)
    ).strftime("%Y-%m-%d")

    items = items_from_pc(bbox=bbox, start_date=start_date, end_date=end_date)
    ds = dataset_from_items(items)
    
    # ds["ndvi"] = (ds["nir"] - ds["red"]) / (ds["nir"] + ds["red"])
    # ds["veg"] = (ds["ndvi"] > 0.4).astype("int16")

    # ds = ds[["veg"]]

    return ds.rio.clip([box(*bbox)], crs="epsg:4326")

In [ ]:
def get_shapefile_mercator_bounds(shapefile_path: str) -> list[float]:
    """
    Get the bounding box (extent) of a shapefile in Web Mercator coordinates.

    Parameters:
    - shapefile_path (str): Path to the shapefile.

    Returns:
    - list[float]: Bounding box as [minx, miny, maxx, maxy] in Web Mercator coordinates.
    """
    # Read the shapefile into a GeoDataFrame
    gdf = gpd.read_file(shapefile_path)

    # Ensure the GeoDataFrame has a valid CRS
    if gdf.crs is None:
        raise ValueError("No CRS information found in the shapefile.")

    # Get the bounding box (extent) as [minx, miny, maxx, maxy]
    bbox = gdf.total_bounds.tolist()
    print(bbox)

    # Convert bounding box to Web Mercator coordinates
    minx, miny = mercantile.xy(bbox[0], bbox[1])
    maxx, maxy = mercantile.xy(bbox[2], bbox[3])

    return [minx, miny, maxx, maxy]

In [ ]:
# AOI
aoi_filepath = "/home/nguyen/GitRepos/swd/ancillary_data/aoi.zip"
aoi_bbox = get_shapefile_mercator_bounds(aoi_filepath)

print(aoi_bbox)


In [ ]:
# Get data base on AOI and period of interest
tile = tile_by_point([8.8, -1.04], zoom=12)
print(tile)
bbox = mercantile.bounds(tile)
print(bbox)

In [ ]:
# Get data
items = items_from_pc(
    # mercantile.bounds(tile),
    aoi_bbox,
    start_date="2016-01-01",
    end_date="2017-01-01",
    collections=PC_S1RTC["collections"],
)
items
# list(items)

ds = dataset_from_items(items)

def reproject_box(bbox, to_crs):
    gds = gpd.GeoSeries([box(*bbox)], crs=4326).to_crs(to_crs)
    return box(*gds.bounds.values[0])

box_reproject = reproject_box(bbox, ds.rio.crs)
# print(box_reproject)

ds = ds.rio.slice_xy(*box_reproject.bounds)
# ds
# ds = ds.rio.clip([box(*bbox)], crs="epsg:4326")
# ds
# ds.isel(time=0).squeeze().rio.to_raster("/media/kirmaier/data/PycharmProjects/sk_tools/swd_pc/s1_rtc.tif")
ds
